In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).to(device).input_ids

    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res


In [17]:
# text = 'What kind of support do you offer to your online students?'
# print(paraphrase(text))

In [24]:
from utils import swinburne_utils

# Load and preprocess the scraped FAQs
faq_data = swinburne_utils.get_faqs()
faq_dataset = [{"question": q, "answer": a} for q, a in faq_data]

/home/phung/Projects/swinburne/technology-design/sample-chat-server/venv/lib/python3.11/site-packages/bs4/builder/__init__.py:314: RuntimeWarning: coroutine 'promise' was never awaited
  for attr in list(attrs.keys()):


In [19]:

for question_answer in faq_dataset:
    question_answer["questions_set"] = paraphrase(question_answer["question"])
    question_answer["answers_set"] = paraphrase(question_answer["answer"])

print(faq_dataset[0])

{'question': 'What support can I expect?', 'answer': 'As a Swinburne Online student, you’ll have support for extended hours, seven days a week, with Student Advisors available to help with anything from tech support to research advice and dedicated online tutors in each of your units. Learn more aboutyour support.', 'questions_set': ['What aid can I receive?', 'Which assistance will be given to me?', 'What help I can get?', 'What kind of assistance can I receive?', 'what support I will be given?'], 'answers_set': ['Swinburne Online students have access to Student Advisors who are available for extended hours, providing all the necessary support they need from tech support to research advice and dedicated online tutors in each unit. Learn more about this.', 'Whether you are a Swinburne Online student or not, there are Student Advisors who offer 24-hour support, including technical assistance and online tutoring for all units. Learn more about what this means.', 'Swinburne Online student